Scraping Olympic medals website

In [1]:
pip install lxml


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import matplotlib.pyplot as plt
import json
import time
import lxml

In [3]:
url = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"
}
session  = requests.Session()
session.get("https://en.wikipedia.org", headers=headers)
time.sleep(2)
response = session.get(url, headers=headers)
data = response.text
soup = bs(data, "html.parser")
olympic_table = soup.find_all('table', class_="wikitable")
n = len(olympic_table)
print(n)

36


In [4]:
for idx, table in enumerate(olympic_table):
    # Extract rows
    rows = []
    for row in table.find_all('tr'):
        cells = row.find_all(['th', 'td'])
        row_data = [cell.get_text(strip=True) for cell in cells]
        rows.append(row_data)

    # Convert to DataFrame
    df = pd.DataFrame(rows)

    # Save to CSV
    filename = f"table_{idx}.csv"
    df.to_csv(filename, index=False, header=False)


In [5]:
table = olympic_table[0]
print(table)

<table class="wikitable sortable sticky-header-multi" style="margin-top:0; text-align:center; font-size:90%;">
<tbody><tr>
<th>Team
</th>
<th align="center" colspan="5" style="background-color:#f2f2ce;"><a href="/wiki/Summer_Olympic_Games" title="Summer Olympic Games">Summer Olympic Games</a>
</th>
<th align="center" colspan="5" style="background-color:#cedff2;"><a href="/wiki/Winter_Olympic_Games" title="Winter Olympic Games">Winter Olympic Games</a>
</th>
<th align="center" colspan="5"><a href="/wiki/Olympic_Games" title="Olympic Games">Combined total</a>
</th></tr>
<tr>
<th><style data-mw-deduplicate="TemplateStyles:r1038841319">.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}</style><span class="rt-commentedText tooltip tooltip-dotted" title="International Olympic Committee country code">Team (IOC code)</span>
</th>
<th style="background-color:#f2f2ce; width:2em;">
<p><link href="mw-data:TemplateStyles:r1038841319" rel="mw-deduplicated-inline-style"><span cla

In [6]:
table = olympic_table[0]

countries = []
summer_gold, summer_silver, summer_bronze, summer_all = [], [], [], []
winter_gold, winter_silver, winter_bronze, winter_all = [], [], [], []
total_gold, total_silver, total_bronze, total = [], [], [], []

for row in table.find_all("tr")[1:]:
    cols = row.find_all(["th", "td"])
    cols = [ele.get_text(strip=True) for ele in cols]
    if len(cols) < 10:
        continue
    meow = row.find("a", href = True)

    countries.append(meow.text if meow else cols[1])
    summer_gold.append(cols[2])
    summer_silver.append(cols[3])
    summer_bronze.append(cols[4])
    summer_all.append(cols[5])
    winter_gold.append(cols[6])
    winter_silver.append(cols[7])
    winter_bronze.append(cols[8])
    winter_all.append(cols[9])
    total_gold.append(cols[10])
    total_silver.append(cols[11])
    total_bronze.append(cols[12])
    total.append(cols[13])

df = pd.DataFrame({
    "Country": countries,
    "Summer Gold": summer_gold,
    "Summer Silver": summer_silver,
    "Summer Bronze": summer_bronze,
    "Summer": summer_all,
    "Winter Gold": winter_gold,
    "Winter Silver": winter_silver,
    "Winter Bronze": winter_bronze,
    "Winter": winter_all,
    "Total Gold": total_gold,
    "Total Silver": total_silver,
    "Total Bronze": total_bronze,
    "Total Medals": total
})

print(df.head)

<bound method NDFrame.head of                               Country Summer Gold Summer Silver Summer Bronze  \
0                                 No.                                           
1                         Afghanistan           0             0             2   
2                             Albania           0             0             2   
3                             Algeria           7             4             9   
4                           Argentina          22            27            31   
..                                ...         ...           ...           ...   
159       Individual Neutral Athletes           1             3             1   
160      Independent Olympic Athletes           1             0             1   
161  Independent Olympic Participants           0             1             2   
162                        Mixed team          11             6             8   
163                                30       5,793         5,764         6,277  

In [7]:
medals = pd.read_csv('table_0.csv')
medals
import re

# Remove everything in brackets (round, square, curly) for column 1 (index 0)
medals.iloc[:, 0] = medals.iloc[:, 0].str.replace(r'\[.*?\]|\(.*?\)|\{.*?\}', '', regex=True)
new_header = medals.iloc[0]  # first row
medals = medals[1:]  # data without the first row
medals.columns = ['Team','SNo.','SGold','SSilver','SBronze','STotal','WNo.','WGold','WSilver','WBronze','WTotal','TNo.','TGold','TSilver','TBronze','TTotal']
medals.to_csv('medals.csv')

In [8]:
df_medals=pd.read_csv("/Users/keerthi/Desktop/Fresh/medals.csv")
len(df)

164

In [9]:
nomedals = pd.read_csv('table_1.csv')

In [10]:
nomedals.head()

,Team (IOC code),No. Summer,No. Winter,No. Games
0,American Samoa(ASA),10,2,12
1,Andorra(AND),13,13,26
2,Angola(ANG),11,0,11
3,Antigua and Barbuda(ANT),12,0,12
4,Aruba(ARU),10,0,10


In [11]:
for col in ['No. Summer', 'No. Winter', 'No. Games']:
    idx = nomedals.columns.get_loc(col) + 1  # Find position to insert after the column
    for i in range(4):
        nomedals.insert(idx + i, f'{col}_extra_{i+1}', 0)

In [12]:
nomedals.columns = ['Team','SNo.','SGold','SSilver','SBronze','STotal','WNo.','WGold','WSilver','WBronze','WTotal','TNo.','TGold','TSilver','TBronze','TTotal']
nomedals.head()

,Team,SNo.,SGold,SSilver,SBronze,STotal,WNo.,WGold,WSilver,WBronze,WTotal,TNo.,TGold,TSilver,TBronze,TTotal
0,American Samoa(ASA),10,0,0,0,0,2,0,0,0,0,12,0,0,0,0
1,Andorra(AND),13,0,0,0,0,13,0,0,0,0,26,0,0,0,0
2,Angola(ANG),11,0,0,0,0,0,0,0,0,0,11,0,0,0,0
3,Antigua and Barbuda(ANT),12,0,0,0,0,0,0,0,0,0,12,0,0,0,0
4,Aruba(ARU),10,0,0,0,0,0,0,0,0,0,10,0,0,0,0


In [13]:
nomedals.iloc[:, 0] = nomedals.iloc[:, 0].str.replace(r'\[.*?\]|\(.*?\)|\{.*?\}', '', regex=True)
nomedals.head()

,Team,SNo.,SGold,SSilver,SBronze,STotal,WNo.,WGold,WSilver,WBronze,WTotal,TNo.,TGold,TSilver,TBronze,TTotal
0,American Samoa,10,0,0,0,0,2,0,0,0,0,12,0,0,0,0
1,Andorra,13,0,0,0,0,13,0,0,0,0,26,0,0,0,0
2,Angola,11,0,0,0,0,0,0,0,0,0,11,0,0,0,0
3,Antigua and Barbuda,12,0,0,0,0,0,0,0,0,0,12,0,0,0,0
4,Aruba,10,0,0,0,0,0,0,0,0,0,10,0,0,0,0


In [14]:
medals_merged = pd.concat([medals, nomedals], ignore_index=True)
medals_merged.to_csv("medals_merged.csv")

In [15]:
df=pd.read_csv("/Users/keerthi/Desktop/Fresh/medals_merged_1.csv")

In [16]:
pip install pycountry


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
df_nomedals=pd.read_csv("/Users/keerthi/Desktop/Fresh/nomedals.csv")

Added a new column with ISO3 country codes for standardized country names.

In [18]:
import pycountry

def get_iso3(country):
    try:
        return pycountry.countries.lookup(country).alpha_3
    except:
        return None

df["ISO3"] = df["Team"].apply(get_iso3)


In [19]:
df_nomedals["ISO3"]=df_nomedals["Team"].apply(get_iso3)

In [20]:
len(df)

234

In [21]:
df.iloc[161]

Unnamed: 0           161
Team          Mixed team
SNo.                   3
SGold                 11
SSilver                6
SBronze                8
STotal                25
WNo.                   0
WGold                  0
WSilver                0
WBronze                0
WTotal                 0
TNo.                   3
TGold                 11
TSilver                6
TBronze                8
TTotal                25
ISO3                None
Name: 161, dtype: object

In [22]:
df.iloc[162]

Unnamed: 0               162
Team          American Samoa
SNo.                      10
SGold                      0
SSilver                    0
SBronze                    0
STotal                     0
WNo.                       2
WGold                      0
WSilver                    0
WBronze                    0
WTotal                     0
TNo.                      12
TGold                      0
TSilver                    0
TBronze                    0
TTotal                     0
ISO3                     ASM
Name: 162, dtype: object

In [23]:
df.head()

,Unnamed: 0,Team,SNo.,SGold,SSilver,SBronze,STotal,WNo.,WGold,WSilver,WBronze,WTotal,TNo.,TGold,TSilver,TBronze,TTotal,ISO3
0,0,Afghanistan,16,0,0,2,2,0,0,0,0,0,16,0,0,2,2,AFG
1,1,Albania,10,0,0,2,2,5,0,0,0,0,15,0,0,2,2,ALB
2,2,Algeria,15,7,4,9,20,3,0,0,0,0,18,7,4,9,20,DZA
3,3,Argentina,26,22,27,31,80,20,0,0,0,0,46,22,27,31,80,ARG
4,4,Armenia,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22,ARM


In [24]:
!pip install dash plotly pandas


In [25]:
pip install nbformat


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [27]:
df.head()

,Unnamed: 0,Team,SNo.,SGold,SSilver,SBronze,STotal,WNo.,WGold,WSilver,WBronze,WTotal,TNo.,TGold,TSilver,TBronze,TTotal,ISO3
0,0,Afghanistan,16,0,0,2,2,0,0,0,0,0,16,0,0,2,2,AFG
1,1,Albania,10,0,0,2,2,5,0,0,0,0,15,0,0,2,2,ALB
2,2,Algeria,15,7,4,9,20,3,0,0,0,0,18,7,4,9,20,DZA
3,3,Argentina,26,22,27,31,80,20,0,0,0,0,46,22,27,31,80,ARG
4,4,Armenia,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22,ARM


Removing the comma for 4 digit numbers and converting them to integers

Adding a new column called Tsummer and Twinter which contains total summer medals and total winter medals respectively 

In [28]:
df = df.iloc[:-1].copy()
df["SGold"] = df["SGold"].str.replace(',', '', regex=False).astype(int)
df["SSilver"] = df["SSilver"].str.replace(',', '', regex=False).astype(int)
df["SBronze"] = df["SBronze"].str.replace(',', '', regex=False).astype(int)
df["WGold"] = df["WGold"].str.replace(',', '', regex=False).astype(int)
df["WSilver"] = df["WSilver"].str.replace(',', '', regex=False).astype(int)
df["WBronze"] = df["WBronze"].str.replace(',', '', regex=False).astype(int)
df["TGold"] = df["TGold"].str.replace(',', '', regex=False).astype(int)
df["TSilver"] = df["TSilver"].str.replace(',', '', regex=False).astype(int)
df["TBronze"] = df["TBronze"].str.replace(',', '', regex=False).astype(int)
df["TTotal"]=df["TTotal"].str.replace(',','',regex=False).astype(int)

df["Tsummer"]=df["SGold"]+df["SSilver"]+df["SBronze"]
df["Twinter"]=df["WGold"]+df["WSilver"]+df["WBronze"]
df.head()

,Unnamed: 0,Team,SNo.,SGold,SSilver,SBronze,STotal,WNo.,WGold,WSilver,WBronze,WTotal,TNo.,TGold,TSilver,TBronze,TTotal,ISO3,Tsummer,Twinter
0,0,Afghanistan,16,0,0,2,2,0,0,0,0,0,16,0,0,2,2,AFG,2,0
1,1,Albania,10,0,0,2,2,5,0,0,0,0,15,0,0,2,2,ALB,2,0
2,2,Algeria,15,7,4,9,20,3,0,0,0,0,18,7,4,9,20,DZA,20,0
3,3,Argentina,26,22,27,31,80,20,0,0,0,0,46,22,27,31,80,ARG,80,0
4,4,Armenia,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22,ARM,22,0


In [29]:
max_gold=df['TGold'].max()
max_gold

np.int64(1219)

In [30]:
df.drop("Unnamed: 0",axis=1)

,Team,SNo.,SGold,SSilver,SBronze,STotal,WNo.,WGold,WSilver,WBronze,WTotal,TNo.,TGold,TSilver,TBronze,TTotal,ISO3,Tsummer,Twinter
0,Afghanistan,16,0,0,2,2,0,0,0,0,0,16,0,0,2,2,AFG,2,0
1,Albania,10,0,0,2,2,5,0,0,0,0,15,0,0,2,2,ALB,2,0
2,Algeria,15,7,4,9,20,3,0,0,0,0,18,7,4,9,20,DZA,20,0
3,Argentina,26,22,27,31,80,20,0,0,0,0,46,22,27,31,80,ARG,80,0
4,Armenia,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22,ARM,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,North Borneo,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,None,0,0
229,Republic of China,3,0,0,0,0,0,0,0,0,0,3,0,0,0,0,None,0,0
230,Saar,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,None,0,0
231,North Yemen,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,None,0,0


Bar graph of the top 25 countries by total medals (Summer + Winter).

In [31]:
top_25_countries = df.nlargest(25, 'TTotal')
fig_total = px.bar(
    top_25_countries,
    x='Team',
    y='TTotal',
    title='Top 25 Countries by Total Olympics Medals',
    labels={'TTotal': 'TTotal', 'Team': 'Team'},
    color='TTotal',
    color_continuous_scale='Blues', #color paletter
    text='TTotal', #display value on the trace,
)


fig_total.update_traces(texttemplate='%{text:,.0f}', textposition='outside') #template of text and location
fig_total.update_layout(title_x=0.5, template='plotly_dark')
fig_total.show()

Bar graph of the bottom 40 countries by total medals (Summer + Winter).

In [32]:
last_40_countries = df.nsmallest(40, 'TTotal')
fig_ltotal = px.bar(
    last_40_countries,
    x='Team',
    y='TTotal',
    title='Last 40 Countries by Total Olympics Medals',
    labels={'TTotal': 'TTotal', 'Team': 'Team'},
    color='TTotal',
    color_continuous_scale='Blues', #color paletter
    text='TTotal', #display value on the trace,
)


fig_ltotal.update_traces(texttemplate='%{text:,.0f}', textposition='outside') #template of text and location
fig_ltotal.update_layout(title_x=0.5, template='plotly_dark')
fig_ltotal.show()

a choropleth map of total medals (Summer + Winter) by country.

In [33]:
fig_t = px.choropleth(
    df,
    locations="ISO3",
    color="TTotal",
    hover_name="ISO3",
    color_continuous_scale="Reds",
    title="Total Olympic Medals by Country",
)
fig_t.update_layout(template="plotly_dark", title_x=0.5)
fig_t.show()


A chloropeth map of total summer medals

In [34]:
fig_ts = px.choropleth(
    df,
    locations="ISO3",
    color="Tsummer",
    hover_name="ISO3",
    color_continuous_scale="Greens",
    title="Total Summer Medals by Country",
)
fig_ts.update_layout(template="plotly_dark", title_x=0.5)
fig_ts.show()


A chloropeth map of total winter medals

In [35]:
fig_tw = px.choropleth(
    df,
    locations="ISO3",
    color="Twinter",
    hover_name="ISO3",
    color_continuous_scale="turbid",
    title="Total Winter Medals by Country",
)
fig_tw.update_layout(template="plotly_dark", title_x=0.5)
fig_tw.show()


A chloropeth map for countries that have 0 medals

In [36]:
fig_nm = px.choropleth(
    df_nomedals,
    locations="ISO3",
    color="TTotal",
    hover_name="ISO3",
    color_continuous_scale="Blues",
    title="Countries that have 0 medals",
)
fig_nm.update_layout(template="plotly_dark", title_x=0.5)
fig_nm.show()


In [37]:
max_summer=df["Tsummer"].max()
max_summer

np.int64(2765)

Subplots of bargraph showing total summer and winter medals 

In [38]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

top_summer_25_countries = df.nlargest(25, 'Tsummer')
top_winter_25_countries = df.nlargest(25, 'Twinter')

fig_sw = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Top 25 Countries by Total Summer Medals", "Top 25 Countries by Total Winter Medals")
)

fig_sw.add_trace(
    go.Bar(
        x=top_summer_25_countries['Team'],
        y=top_summer_25_countries['Tsummer'],
        marker_color='gold',
        text=top_summer_25_countries['Tsummer'],
        textposition='outside',
        name='Summer'
    ),
    row=1, col=1
)

fig_sw.add_trace(
    go.Bar(
        x=top_winter_25_countries['Team'],
        y=top_winter_25_countries['Twinter'],
        marker_color='silver',
        text=top_winter_25_countries['Twinter'],
        textposition='outside',
        name='Winter'
    ),
    row=1, col=2
)
fig_sw.update_layout(
    title_text="Top 25 Countries by Summer and Winter Olympic Medals",
    title_x=0.5,
    template='plotly_dark',
    showlegend=False
)

fig_sw.show()


In [39]:
from plotly.subplots import make_subplots

Subplots of bar graphs showing total Gold, Silver, and Bronze medals by country.

In [40]:
top25 = df.nlargest(25, 'TGold')
fig_mixed = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "Top 25 - Total Gold Medals",
        "Top 25 - Total Silver Medals",
        "Top 25 - Total Bronze Medals"
    ]
)
fig_mixed.add_trace(
    go.Bar(
        x=top25['Team'],
        y=top25['TGold'],
        text=top25['TGold'],
        textposition='outside',
        marker=dict(color=top25['TGold'], colorscale='Blues')
    ),
    row=1, col=1
)
fig_mixed.add_trace(
    go.Bar(
        x=top25['Team'],
        y=top25['TSilver'],
        text=top25['TSilver'],
        textposition='outside',
        marker=dict(color=top25['TSilver'], colorscale='Greens')
    ),
    row=1, col=2
)
fig_mixed.add_trace(
    go.Bar(
        x=top25['Team'],
        y=top25['TBronze'],
        text=top25['TBronze'],
        textposition='outside',
        marker=dict(color=top25['TBronze'], colorscale='Oranges')
    ),
    row=1, col=3
)
fig_mixed.update_layout(
    height=600,
    width=1800,
    template='plotly_dark',
    title_text="Top 25 Countries – Gold, Silver & Bronze",
    title_x=0.5
)

fig_mixed.show()


Subplots of bar graphs showing Summer Gold, Silver, and Bronze medals by country.

In [41]:
top25 = df.nlargest(25, 'SGold')
fig_mixed = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "Top 25 - Summer Gold Medals",
        "Top 25 - Summer Silver Medals",
        "Top 25 - Summer Bronze Medals"
    ]
)
fig_mixed.add_trace(
    go.Bar(
        x=top25['Team'],
        y=top25['SGold'],
        text=top25['SGold'],
        textposition='outside',
        marker=dict(color=top25['SGold'], colorscale='Blues')
    ),
    row=1, col=1
)
fig_mixed.add_trace(
    go.Bar(
        x=top25['Team'],
        y=top25['SSilver'],
        text=top25['SSilver'],
        textposition='outside',
        marker=dict(color=top25['SSilver'], colorscale='Greens')
    ),
    row=1, col=2
)
fig_mixed.add_trace(
    go.Bar(
        x=top25['Team'],
        y=top25['SBronze'],
        text=top25['SBronze'],
        textposition='outside',
        marker=dict(color=top25['SBronze'], colorscale='Oranges')
    ),
    row=1, col=3
)
fig_mixed.update_layout(
    height=600,
    width=1800,
    template='plotly_dark',
    title_text="Top 25 Countries – Summer Gold, Summer Silver & Summer Bronze",
    title_x=0.5
)

fig_mixed.show()


Subplots of bar graphs showing Winter Gold, Silver, and Bronze medals by country.

In [42]:
top25 = df.nlargest(25, 'WGold')
fig_mixed = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "Top 25 - Winter Gold Medals",
        "Top 25 - Winter Silver Medals",
        "Top 25 - Winter Bronze Medals"
    ]
)
fig_mixed.add_trace(
    go.Bar(
        x=top25['Team'],
        y=top25['WGold'],
        text=top25['WGold'],
        textposition='outside',
        marker=dict(color=top25['WGold'], colorscale='Blues')
    ),
    row=1, col=1
)
fig_mixed.add_trace(
    go.Bar(
        x=top25['Team'],
        y=top25['WSilver'],
        text=top25['WSilver'],
        textposition='outside',
        marker=dict(color=top25['WSilver'], colorscale='Greens')
    ),
    row=1, col=2
)
fig_mixed.add_trace(
    go.Bar(
        x=top25['Team'],
        y=top25['WBronze'],
        text=top25['WBronze'],
        textposition='outside',
        marker=dict(color=top25['WBronze'], colorscale='Oranges')
    ),
    row=1, col=3
)
fig_mixed.update_layout(
    height=600,
    width=1800,
    template='plotly_dark',
    title_text="Top 25 Countries – Winter Gold, Winter Silver & Winter Bronze",
    title_x=0.5
)

fig_mixed.show()


Bar graph of the last 40 countries by total Summer medals

In [43]:
last_summer_40_countries = df.nsmallest(40, 'Tsummer')
fig_lsummer = px.bar(
    last_summer_40_countries,
    x='Team',
    y='Tsummer',
    title='last 40 Countries by Total summer Olympics Medals',
    labels={'Tsummer': 'Total summer medals', 'Team': 'Country'},
    color='Tsummer',
    color_continuous_scale='Blues', #color paletter
    text='Tsummer', #display value on the trace,
)


fig_lsummer.update_traces(texttemplate='%{text:,.0f}', textposition='outside') #template of text and location
fig_lsummer.update_layout(title_x=0.5, template='plotly_dark')
fig_lsummer.show()

Bar graph of the last 40 countries by total winter medals

In [44]:
last_winter_40_countries = df.nsmallest(40, 'Twinter')
fig_lwinter = px.bar(
    last_winter_40_countries,
    x='Team',
    y='Twinter',
    title='last 40 Countries by Total summer Olympics Medals',
    labels={'Twinter': 'Total winter medals', 'Team': 'Country'},
    color='Twinter',
    color_continuous_scale='Blues', #color paletter
    text='Twinter', #display value on the trace,
)


fig_lwinter.update_traces(texttemplate='%{text:,.0f}', textposition='outside') #template of text and location
fig_lwinter.update_layout(title_x=0.5, template='plotly_dark')
fig_lwinter.show()

In [45]:
!pip install dash plotly pandas dash-bootstrap-components

In [46]:
# Import required libraries
from dash import Dash, html, dcc, Input, Output, State, callback
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [47]:
from IPython.display import HTML

In [48]:
top_summer_25_countries = df.nlargest(25, 'Tsummer')
fig_tsummer = px.bar( 
    top_summer_25_countries, 
    x='Team',
    y='Tsummer',
    title='Top 25 Countries by Total summer Olympics Medals', 
    labels={'Tsummer': 'Total summer medals', 'Team': 'Team'}, 
    color='Tsummer', color_continuous_scale='Blues',
    text='Tsummer',
)
fig_tsummer.update_traces(texttemplate='%{text:,.0f}',textposition='outside'),
fig_tsummer.update_layout(title_x=0.5, template='plotly_dark')
fig_tsummer.show()



In [49]:
top_winter_25_countries = df.nlargest(25, 'Twinter')
fig_twinter = px.bar( 
    top_winter_25_countries, 
    x='Team', 
    y='Twinter', 
    title='Top 25 Countries by Total Winter Olympics Medals', 
    labels={'Twinter': 'Total summer medals', 'Team': 'Team'}, 
    color='Twinter', 
    color_continuous_scale='Blues', 
    text='Twinter', 
 )
fig_twinter.update_traces(texttemplate='%{text:,.0f}', textposition='outside') 
fig_twinter.update_layout(title_x=0.5, template='plotly_dark') 
fig_twinter.show()

In [50]:
summer_html = fig_tsummer.to_html(full_html=False)
winter_html = fig_twinter.to_html(full_html=False)


Building Dashboard

In [51]:
pip install dash plotly pandas dash-bootstrap-components


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
from dash import Dash,html,dcc,Input,Output,State,callback
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


Dashboard with a single markdown (Names of Countries)

In [ ]:
'''
external_stylesheets = [
    dbc.themes.BOOTSTRAP,
    "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css"
]

app = Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(
            html.Div([
                html.H1([
                    html.I(className="fa-solid fa-medal", style={'marginRight': '14px'}),
                    "Olympics Medals Dashboard"
                ], style={'color': 'white', 'margin': 0, 'fontWeight': 'bold'}),

                html.P(
                    "Analysis of Olympics medals of different countries",
                    style={'color': 'white', 'opacity': 0.9, 'margin': 0}
                )
            ],
            style={
                'background': 'linear-gradient(135deg, #667eea 0%, #764ba2 100%)',
                'padding': '30px',
                'borderRadius': '10px',
                'marginBottom': '30px',
            })
        )
    ]),
    dbc.Row([
        dbc.Col(
            html.Div([
                html.Div(html.I(className="fa-solid fa-globe"),
                         style={'fontSize': '40px', 'color': '#3b82f6', 'marginBottom': '10px'}),

                html.Div(df['Team'].nunique(),
                         style={'fontSize': '32px', 'fontWeight': 'bold', 'color': '#3b82f6'}),

                html.Div("Total Countries", style={'color': '#666', 'fontSize': '18px'})
            ],
            style={
                'background': 'white', 'padding': '25px', 'borderRadius': '10px',
                'boxShadow': '0 2px 8px rgba(0,0,0,0.1)', 'textAlign': 'center'
            }),
            width=6
        ),
        dbc.Col(
            html.Div([
                html.Div(html.I(className="fa-solid fa-medal"),
                         style={'fontSize': '40px', 'color': '#10b981', 'marginBottom': '10px'}),

                html.Div(df['TTotal'].max(),
                         style={'fontSize': '32px', 'fontWeight': 'bold', 'color': '#10b981'}),

                html.Div("Total Medals", style={'color': '#666', 'fontSize': '18px'})
            ],
            style={
                'background': 'white', 'padding': '25px', 'borderRadius': '10px',
                'boxShadow': '0 2px 8px rgba(0,0,0,0.1)', 'textAlign': 'center'
            }),
            width=6
        ),

    ], style={'marginBottom': '30px'}),
    dbc.Row([
        dbc.Col(
            html.Div([
                html.Div(html.I(className="fa-solid fa-medal", style={'color': 'gold'}),
                         style={'fontSize': '40px', 'marginBottom': '10px'}),

                html.Div(df['TGold'].max(),
                         style={'fontSize': '32px', 'fontWeight': 'bold', 'color': '#f59e0b'}),

                html.Div("Total Gold Medals", style={'color': '#666', 'fontSize': '18px'})
            ],
            style={
                'background': 'white', 'padding': '25px', 'borderRadius': '10px',
                'boxShadow': '0 2px 8px rgba(0,0,0,0.1)', 'textAlign': 'center'
            }),
            width=4
        ),
        dbc.Col(
            html.Div([
                html.Div(html.I(className="fa-solid fa-medal", style={'color': 'silver'}),
                         style={'fontSize': '40px', 'marginBottom': '10px'}),

                html.Div(df['TSilver'].max(),
                         style={'fontSize': '32px', 'fontWeight': 'bold', 'color': '#8b5cf6'}),

                html.Div("Total Silver Medals", style={'color': '#666', 'fontSize': '18px'})
            ],
            style={
                'background': 'white', 'padding': '25px', 'borderRadius': '10px',
                'boxShadow': '0 2px 8px rgba(0,0,0,0.1)', 'textAlign': 'center'
            }),
            width=4
        ),
        dbc.Col(
            html.Div([
                html.Div(html.I(className="fa-solid fa-medal", style={'color': '#cd7f32'}),
                         style={'fontSize': '40px', 'marginBottom': '10px'}),

                html.Div(df['TBronze'].max(),
                         style={'fontSize': '32px', 'fontWeight': 'bold', 'color': '#f59e0b'}),

                html.Div("Maximum Bronze Medals", style={'color': '#666', 'fontSize': '18px'})
            ],
            style={
                'background': 'white', 'padding': '25px', 'borderRadius': '10px',
                'boxShadow': '0 2px 8px rgba(0,0,0,0.1)', 'textAlign': 'center'
            }),
            width=4
        ),

    ], style={'marginBottom': '30px'}),

    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig_tsummer), width=6),
        dbc.Col(dcc.Graph(figure=fig_twinter), width=6),
    ], style={'marginBottom': '30px'}),

    dbc.Row([
        dbc.Col([
            html.Div([
                html.Label("Select Country:", style={'fontWeight': 'bold', 'color': 'white'}),
                dcc.Dropdown(
                    id='country-select',
                    options=[{'label': c, 'value': c} for c in df['Team'].unique()],
                    value='United States',
                    clearable=False,
                    style={'marginBottom': '20px'}
                ),
                dcc.Graph(id='country-medal-bar')
            ],
            style={
                'background': '#111',
                'padding': '20px',
                'borderRadius': '10px',
                'boxShadow': '0 2px 8px rgba(0,0,0,0.3)'
            })
        ], width=12)
    ], style={'marginBottom': '30px'}),

], fluid=True)
@app.callback(
    Output('country-medal-bar', 'figure'),
    Input('country-select', 'value')
)
def update_country_bar(selected_country):

    row = df[df['Team'] == selected_country].iloc[0]

    medals = {
        'Summer Gold': row['SGold'],
        'Summer Silver': row['SSilver'],
        'Summer Bronze': row['SBronze'],
        'Winter Gold': row['WGold'],
        'Winter Silver': row['WSilver'],
        'Winter Bronze': row['WBronze'],
        'Total Gold': row['TGold'],
        'Total Silver': row['TSilver'],
        'Total Bronze': row['TBronze']
    }

    plot_df = pd.DataFrame({
        'Medal Type': list(medals.keys()),
        'Count': list(medals.values())
    })

    fig = px.bar(
        plot_df,
        x="Medal Type",
        y="Count",
        text="Count",
        color="Count",
        color_continuous_scale="Blues",
        title=f"Olympic Medals for {selected_country}"
    )

    fig.update_traces(textposition='outside')
    fig.update_layout(title_x=0.5, template="plotly_dark", height=450)

    return fig
if __name__ == "__main__":
    app.run(debug=True, open_browser=True)


'''

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/Users/keerthi/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
'''from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go

app = Dash(
    __name__,
    external_stylesheets=[
        dbc.themes.BOOTSTRAP,
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css"
    ]
)

# -----------------------------------------------------
# GLOBAL DARK THEME FOR THE ENTIRE PAGE
# -----------------------------------------------------
PAGE_STYLE = {
    "backgroundColor": "#000000",   # Black background
    "color": "white",
    "padding": "20px"
}

CARD_STYLE = {
    "backgroundColor": "white",
    "borderRadius": "16px",
    "padding": "20px",
    "boxShadow": "0px 4px 12px rgba(255,255,255,0.1)"
}

TAB_STYLE = {
    "backgroundColor": "#111111",
    "color": "white",
    "padding": "12px",
    "border": "none"
}

TAB_SELECTED_STYLE = {
    "backgroundColor": "#222222",
    "color": "white",
    "padding": "12px",
    "border": "none"
}

# Make all graphs dark-themed
def dark_fig(fig):
    fig.update_layout(
        plot_bgcolor="#000000",
        paper_bgcolor="#000000",
        font_color="white"
    )
    return fig

app.layout = dbc.Container([
    html.Div([
        html.H1("Olympics Medals Dashboard",
                style={
                    "color": "white",
                    "fontWeight": "bold",
                    "margin": "0",
                    "padding": "10px 0"
                }),
        html.P("Analysis of Olympics medals of different countries",
               style={"color": "white", "margin": "0"})
    ], style={
        "background": "linear-gradient(90deg, #667eea 0%, #764ba2 100%)",
        "padding": "30px",
        "borderRadius": "20px",
        "marginBottom": "30px"
    }),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-globe fa-3x", style={"color": "#4f8df7"}),
                    html.H3("162", style={"color": "#4f8df7"}),
                    html.P("Total Countries", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=6),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-medal fa-3x", style={"color": "#22c55e"}),
                    html.H3("3095", style={"color": "#22c55e"}),
                    html.P("Maximum Total Medals", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=6),
    ], className="mb-4"),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-medal fa-3x", style={"color": "#fbbf24"}),
                    html.H3("1219", style={"color": "#fbbf24"}),
                    html.P("Maximum Gold Medals", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=4),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-medal fa-3x", style={"color": "#c4c4c4"}),
                    html.H3("1000", style={"color": "#a78bfa"}),
                    html.P("Maximum Silver Medals", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=4),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-medal fa-3x", style={"color": "#cd7f32"}),
                    html.H3("876", style={"color": "#f59e0b"}),
                    html.P("Maximum Bronze Medals", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=4),
    ], className="mb-5"),
    dcc.Tabs(id="tabs", value="tab-1",
             style={"marginBottom": "20px"},
             children=[
                 dcc.Tab(label="Overview", value="tab-1",
                         style=TAB_STYLE,
                         selected_style=TAB_SELECTED_STYLE,
                         children=[
                             html.Br(),
                             dcc.Graph(id="total-graph"),
                             html.Br(),
                             dcc.Graph(id="summer-graph"),
                             html.Br(),
                             dcc.Graph(id="winter-graph"),
                         ]),

                 dcc.Tab(label="Country Comparison", value="tab-2",
                         style=TAB_STYLE,
                         selected_style=TAB_SELECTED_STYLE,
                         children=[
                             html.Br(),
                             html.Label("Select Country 1"),
                             dcc.Dropdown(
                                 id="country1",
                                 options=[{"label": c, "value": c} for c in df["Team"].unique()],
                                 value=df["Team"].unique()[0]
                             ),
                             html.Br(),
                             html.Label("Select Country 2"),
                             dcc.Dropdown(
                                 id="country2",
                                 options=[{"label": c, "value": c} for c in df["Team"].unique()],
                                 value=df["Team"].unique()[1]
                             ),
                             html.Br(),
                             dcc.Graph(id="compare-graph")
                         ]),

                 dcc.Tab(label="Medal Distribution", value="tab-3",
                         style=TAB_STYLE,
                         selected_style=TAB_SELECTED_STYLE,
                         children=[
                             html.Br(),
                             html.Label("Select Country"),
                             dcc.Dropdown(
                                 id="country-dist",
                                 options=[{"label": c, "value": c} for c in df["Team"].unique()],
                                 value=df["Team"].unique()[0]
                             ),
                             html.Br(),
                             dcc.Graph(id="dist-graph")
                         ]),
             ])
], fluid=True, style=PAGE_STYLE)
@app.callback(
    Output("total-graph", "figure"),
    Output("summer-graph", "figure"),
    Output("winter-graph", "figure"),
    Input("tabs", "value")
)
def update_overview(tab):
    if tab != "tab-1":
        return {}, {}, {}

    fig_total = dark_fig(px.bar(df.nlargest(25, "TTotal"), x="Team", y="TTotal",
                                title="Top 25 Countries — Total Medals",
                                color="TTotal", color_continuous_scale="Viridis"))

    fig_summer = dark_fig(px.bar(df.nlargest(25, "TGold"), x="Team", y="TGold",
                                 title="Top 25 Summer — Gold Medals",
                                 color="TGold", color_continuous_scale="Oranges"))

    fig_winter = dark_fig(px.bar(df.nlargest(25, "TSilver"), x="Team", y="TSilver",
                                 title="Top 25 Winter — Silver Medals",
                                 color="TSilver", color_continuous_scale="Blues"))

    return fig_total, fig_summer, fig_winter


@app.callback(
    Output("compare-graph", "figure"),
    Input("country1", "value"),
    Input("country2", "value")
)
def update_compare(c1, c2):
    subset = df[df["Team"].isin([c1, c2])]
    fig = px.bar(subset, x="Team", y=["TGold", "TSilver", "TBronze"],
                 barmode="group",
                 title=f"Medal Comparison: {c1} vs {c2}")
    return dark_fig(fig)


@app.callback(
    Output("dist-graph", "figure"),
    Input("country-dist", "value")
)
def update_dist(country):
    row = df[df["Team"] == country].iloc[0]
    fig = px.pie(
        names=["Summer Gold", "Summer Silver", "Summer Bronze",
               "Winter Gold", "Winter Silver", "Winter Bronze"],
        values=[row["SGold"], row["SSilver"], row["SBronze"],
                row["WGold"], row["WSilver"], row["WBronze"]],
        title=f"{country} Medal Distribution"
    )
    return dark_fig(fig)


app.run(debug=True, port=8085)
'''

'from dash import Dash, html, dcc, Input, Output\nimport dash_bootstrap_components as dbc\nimport plotly.express as px\nimport plotly.graph_objects as go\n\napp = Dash(\n    __name__,\n    external_stylesheets=[\n        dbc.themes.BOOTSTRAP,\n        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css"\n    ]\n)\n\n# -----------------------------------------------------\n# GLOBAL DARK THEME FOR THE ENTIRE PAGE\n# -----------------------------------------------------\nPAGE_STYLE = {\n    "backgroundColor": "#000000",   # Black background\n    "color": "white",\n    "padding": "20px"\n}\n\nCARD_STYLE = {\n    "backgroundColor": "white",\n    "borderRadius": "16px",\n    "padding": "20px",\n    "boxShadow": "0px 4px 12px rgba(255,255,255,0.1)"\n}\n\nTAB_STYLE = {\n    "backgroundColor": "#111111",\n    "color": "white",\n    "padding": "12px",\n    "border": "none"\n}\n\nTAB_SELECTED_STYLE = {\n    "backgroundColor": "#222222",\n    "color": "white",\n    "padding"

Dashboard with 3 tabs each showing Overview ,Comparison between two countries ,Individual medals count respectively

In [54]:
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go

app = Dash(
    __name__,
    external_stylesheets=[
        dbc.themes.BOOTSTRAP,
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css"
    ]
)
PAGE_STYLE = {
    "backgroundColor": "#000000", 
    "color": "white",
    "padding": "20px"
}

CARD_STYLE = {
    "backgroundColor": "white",
    "borderRadius": "16px",
    "padding": "20px",
    "boxShadow": "0px 4px 12px rgba(255,255,255,0.1)"
}

TAB_STYLE = {
    "backgroundColor": "#111111",
    "color": "white",
    "padding": "12px",
    "border": "none"
}

TAB_SELECTED_STYLE = {
    "backgroundColor": "#222222",
    "color": "white",
    "padding": "12px",
    "border": "none"
}
def dark_fig(fig):
    fig.update_layout(
        plot_bgcolor="#000000",
        paper_bgcolor="#000000",
        font_color="white"
    )
    return fig

app.layout = dbc.Container([
    html.Div([
        html.H1("Olympics Medals Dashboard",
                style={
                    "color": "white",
                    "fontWeight": "bold",
                    "margin": "0",
                    "padding": "10px 0"
                }),
        html.P("Analysis of Olympics medals of different countries",
               style={"color": "white", "margin": "0"})
    ], style={
        "background": "linear-gradient(90deg, #667eea 0%, #764ba2 100%)",
        "padding": "30px",
        "borderRadius": "20px",
        "marginBottom": "30px"
    }),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-globe fa-3x", style={"color": "#4f8df7"}),
                    html.H3((df['Team'].nunique()-1), style={"color": "#4f8df7"}),
                    html.P("Total Countries", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=6),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-medal fa-3x", style={"color": "#22c55e"}),
                    html.H3(df['TTotal'].max(), style={"color": "#22c55e"}),
                    html.P("Maximum Total Medals", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=6),
    ], className="mb-4"),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-medal fa-3x", style={"color": "#fbbf24"}),
                    html.H3(df['TGold'].max(), style={"color": "#fbbf24"}),
                    html.P("Maximum Gold Medals", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=4),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-medal fa-3x", style={"color": "#c4c4c4"}),
                    html.H3(df['TSilver'].max(), style={"color": "#a78bfa"}),
                    html.P("Maximum Silver Medals", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=4),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.I(className="fas fa-medal fa-3x", style={"color": "#cd7f32"}),
                    html.H3(df['TBronze'].max(), style={"color": "#f59e0b"}),
                    html.P("Maximum Bronze Medals", className="text-muted")
                ])
            ], style=CARD_STYLE, className="text-center")
        ], width=4),
    ], className="mb-5"),
    dcc.Tabs(id="tabs", value="tab-1",
             style={"marginBottom": "20px"},
             children=[
                 dcc.Tab(label="Overview", value="tab-1",
                         style=TAB_STYLE,
                         selected_style=TAB_SELECTED_STYLE,
                         children=[
                             html.Br(),
                             dcc.Graph(id="total-graph"),
                             html.Br(),
                             dcc.Graph(id="summer-graph"),
                             html.Br(),
                             dcc.Graph(id="winter-graph"),
                         ]),

                 dcc.Tab(
                        label="Country Comparison",
                        value="tab-2",
                        style=TAB_STYLE,
                        selected_style=TAB_SELECTED_STYLE,
                        children=[
                            html.Br(),
                            html.Label("Select Country 1", style={"color": "white"}), 
                            dcc.Dropdown(
                                id="country1",
                                options=[{"label": c, "value": c} for c in df["Team"].unique()],
                                value=df["Team"].unique()[0],
                                clearable=False,
                                style={
                                    "backgroundColor": "white",
                                    "color": "black",        
                                    "borderRadius": "8px",
                                    "marginBottom": "15px"
                                }
                            ),
                            html.Label("Select Country 2", style={"color": "white"}),
                            dcc.Dropdown(
                                id="country2",
                                options=[{"label": c, "value": c} for c in df["Team"].unique()],
                                value=df["Team"].unique()[1],
                                clearable=False,
                                style={
                                    "backgroundColor": "white",
                                    "color": "black",       
                                    "borderRadius": "8px",
                                    "marginBottom": "20px"
                            },
                        ),
        dcc.Graph(id="compare-graph")
    ]
),

                 dcc.Tab(label="Medal Distribution", value="tab-3",
                         style=TAB_STYLE,
                         selected_style=TAB_SELECTED_STYLE,
                         children=[
                             html.Br(),
                             html.Label("Select Country"),
                             dcc.Dropdown(
                                 id="country-dist",
                                 options=[{"label": c, "value": c} for c in df["Team"].unique()],
                                 value="United States",
                                 clearable=False,
                                 style={
                                    "backgroundColor": "white",
                                    "color": "black",       
                                    "borderRadius": "8px",
                                    "marginBottom": "20px"
                                 },
                             ),
                             html.Br(),
                             dcc.Graph(id="dist-graph")
                         ]),
             ])
], fluid=True, style=PAGE_STYLE)
@app.callback(
    Output("total-graph", "figure"),
    Output("summer-graph", "figure"),
    Output("winter-graph", "figure"),
    Input("tabs", "value")
)
def update_overview(tab):
    if tab != "tab-1":
        return {}, {}, {}

    fig_total = dark_fig(px.bar(df.nlargest(25, "TTotal"), x="Team", y="TTotal",
                                title="Top 25 Countries — Total Medals",
                                color="TTotal", color_continuous_scale="Viridis"))

    fig_summer = dark_fig(px.bar(df.nlargest(25, "TGold"), x="Team", y="TGold",
                                 title="Top 25 Summer — Gold Medals",
                                 color="TGold", color_continuous_scale="Oranges"))

    fig_winter = dark_fig(px.bar(df.nlargest(25, "TSilver"), x="Team", y="TSilver",
                                 title="Top 25 Winter — Silver Medals",
                                 color="TSilver", color_continuous_scale="Blues"))

    return fig_total, fig_summer, fig_winter


@app.callback(
    Output("compare-graph", "figure"),
    Input("country1", "value"),
    Input("country2", "value")
)
def update_compare(c1, c2):
    subset = df[df["Team"].isin([c1, c2])]
    fig = px.bar(subset, x="Team", y=["TGold", "TSilver", "TBronze"],
                 barmode="group",
                 title=f"Medal Comparison: {c1} vs {c2}")
    return dark_fig(fig)


@app.callback(
    Output("dist-graph", "figure"),
    Input("country-dist", "value")
)


def update_dist(country):
    row = df[df["Team"] == country].iloc[0]
    fig = make_subplots(
        rows=1, cols=3,
        specs=[[{"type": "domain"}, {"type": "domain"}, {"type": "domain"}]],
        subplot_titles=[
            "Summer Medals", "Winter Medals", "Season Totals"
        ]
    )
    fig.add_trace(go.Pie(
        labels=["Gold", "Silver", "Bronze"],
        values=[row["SGold"], row["SSilver"], row["SBronze"]],
        name="Summer",
        hole=0.4,
        textinfo="label+value"
    ), 1, 1)
    fig.add_trace(go.Pie(
        labels=["Gold", "Silver", "Bronze"],
        values=[row["WGold"], row["WSilver"], row["WBronze"]],
        name="Winter",
        hole=0.4,
        textinfo="label+value"
    ), 1, 2)
    summer_total = row["SGold"] + row["SSilver"] + row["SBronze"]
    winter_total = row["WGold"] + row["WSilver"] + row["WBronze"]

    fig.add_trace(go.Pie(
        labels=["Total medals","Summer Total", "Winter Total"],
        values=[row["TTotal"],row["Tsummer"], row["Twinter"]],
        name="Totals",
        hole=0.4,
        textinfo="label+value"
    ), 1, 3)
    fig.update_layout(
        title_text=f"{country} Medal Distribution Overview",
        showlegend=True,
        paper_bgcolor="#000000",
        plot_bgcolor="#000000",
        font_color="white",
        annotations=[dict(font_size=14, showarrow=False)],
        height=500
    )

    return fig



app.run(debug=True, port=8085)
